In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
import time
import math
import pickle


def load_json_list(path, verbose=False):
    if verbose:
        print(f"Loading file: {path}")
    with open(path, "rb") as infile:
        json_list = json.load(infile)
    return json_list

def save_json(path, data):
    with open(path, "w", encoding="utf-8") as outfile:
        json.dump(data, outfile, ensure_ascii=False)

In [2]:
os.chdir("..")

In [69]:
completed_search_files = [f for f in os.listdir("data/nu-api-data-raw/")]
completed_search_files.sort()
number_of_pages = {}
for file in completed_search_files:
    search = load_json_list(f"data/nu-api-data-raw/{file}")
    info = {"n_success": math.ceil(len(search) / 50) if search else 0}
    number_of_pages[file] = info
    print(f"{file}: {info}")
    

Afghanistan-Uzbek?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 4}
Algeria-Berber?Date_gt_1991-01-01_and_Date_lt_1999-12-31.json: {'n_success': 9}
Algeria-Berber?Date_gt_2011-01-01_and_Date_lt_2015-12-31.json: {'n_success': 7}
Azerbaijan-Lezgin?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 4}
Bangladesh-ChittagongHill?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 26}
Belgium-FlemishORFleming?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 0}
Belgium-Walloon?Date_gt_1991-01-01_and_Date_lt_1999-12-31.json: {'n_success': 6}
Belgium-Walloon?Date_gt_2011-01-01_and_Date_lt_2015-12-31.json: {'n_success': 11}
Bhutan-LhotshampaORLhotsampa?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 3}
Brazil-IndigenousPeople?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 0}
Cameroon-West?Date_gt_1991-01-01_and_Date_lt_2015-12-31.json: {'n_success': 348}
Canada-IndigenousPeople?Date_gt_1991-01-01_and_Date_lt_2015-12-31

In [70]:
complete_search_list = load_json_list("data/dispute_searches-complete.json")
incomplete_idx = []
for idx, search in enumerate(complete_search_list):
    file_name = search["name"] + '?' + search["date"].replace(' ', '_') + '.json'
    if "n_requests" not in search:
        next
    else:
        if number_of_pages[file_name]["n_success"] != search["n_requests"]:
            incomplete_idx.append((idx, number_of_pages[file_name]["n_success"], search["n_requests"]))
        
incomplete_idx

[(36, 0, 15),
 (59, 29, 43),
 (60, 0, 44),
 (61, 0, 47),
 (62, 0, 51),
 (63, 0, 54),
 (64, 0, 59),
 (65, 0, 66),
 (66, 0, 69),
 (67, 0, 74),
 (68, 0, 74),
 (69, 0, 90),
 (70, 0, 101),
 (71, 0, 102),
 (72, 0, 108),
 (73, 0, 109),
 (74, 0, 109),
 (75, 0, 113),
 (76, 0, 128),
 (77, 0, 136),
 (78, 0, 143),
 (79, 0, 144),
 (80, 0, 152),
 (81, 0, 160),
 (82, 0, 163),
 (83, 0, 178),
 (84, 0, 180),
 (85, 0, 216),
 (86, 0, 223),
 (87, 0, 242),
 (88, 0, 243),
 (89, 0, 245),
 (90, 0, 246),
 (91, 0, 290),
 (92, 0, 303),
 (93, 0, 339),
 (95, 172, 349),
 (97, 367, 368),
 (101, 1, 510),
 (102, 0, 547),
 (103, 1, 557),
 (104, 6, 699),
 (105, 0, 790),
 (106, 0, 821),
 (107, 3, 849),
 (108, 33, 853),
 (109, 4, 854),
 (110, 2, 873),
 (111, 5, 880)]

In [106]:
failed_searches = [complete_search_list[i] for i in list(np.array(incomplete_idx)[:, 0])]
successful_searches = [complete_search_list[i] for i in range(len(complete_search_list)) if i not in list(np.array(incomplete_idx)[:, 0])]

In [92]:
# Overwrite complete searces with successful searches
save_json(path='data/dispute_searches-complete.json', data=successful_searches)

In [107]:
# Append failed searches to incomplete searches
incomplete_searches = load_json_list(path='data/dispute_searches-incomplete.json')
failed_searches.reverse()
incomplete_searches.extend(failed_searches)
save_json(path='data/dispute_searches-incomplete.json', data=incomplete_searches)